# MTH8999 : Analyse des valeurs extrêmes en environnement

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 1 à 4 - Applications

Applications avec des données réelles. Vous pouvez utiliser n'importe laquelle des méthodes vues en classe, ou même une combinaison de celles-ci, afin de comparer les résultats.


In [1]:
using CSV, DataFrames, Dates, Distributions, Extremes, Gadfly

La fonction suivante est utilse pour partitionner en blocs, mais si le nombre de données est non divisible par la taille de bloc désirée.

In [2]:
"""
    blockpartition(x::AbstractArray{<:Real}, m::Int)

Paritionne en matrice à m `lignes` le vecteur `x`.

##Détail

Si le vecteur `x` ne se divise pas en `m`, les premières valeurs de `x` sont rejetées.
"""
function blockpartition(x::AbstractArray{<:Real}, m::Int)
    
    n = length(x)
    
    quotient = floor(Int64, n/m)
    reste = n % m
    
    y = x[reste+1:end]
    
    return reshape(y, m, quotient)
    
end

blockpartition

# Exercice 1 - Débits de la rivière Chaudière

Le fichier chaudière.csv contient les maxima annuels des débits en m³/s de la rivère Chaudière de 1961 à 2020.

À partir de ces données, 

1. Estimez le débit qui est dépassé en moyenne une fois aux 100 ans.
2. Estimez la période de retour de 2500 m³/s. Autement dit, combien d'années en moyenne il faut attendre pour observer un débit excédent 2500 m³/s ?
3. Estimez le débit qui ne pourrait jamais être dépassé.

In [3]:
data = CSV.read("chaudiere.csv", DataFrame)

first(data, 5)

Row,Year,Discharge
,Int64,Float64
1,1961,851.452
2,1962,784.852
3,1963,1224.8
4,1964,1235.38
5,1965,591.892


# Exercice 2 - Niveaux de la mer à Fremantle

Le fichier fremantle.csv contient les maxima annuels des niveaux de la mer en m à Fremantle en Australie de 1897 à 2022

À partir de ces données, 

1. Estimez le niveau qui est dépassé en moyenne une fois aux 100 ans.
2. Estimez la probabilité que le niveau n'excède pas 6.9 m en 2025 ?

In [4]:
data = CSV.read("fremantle.csv", DataFrame)

first(data, 5)

Row,Year,SeaLevel
,Int64,Float64
1,1897,6.569
2,1898,6.641
3,1901,6.618
4,1903,6.61
5,1904,6.648


# Exercice 3 - Températures maximales journalières à Montréal

Le fichier tasmax.csv contient les températures maximales journalières en Celcius à Montréal de 1942 à 2025.

À partir de ces données, 

1. Estimez la temprature qui est dépassée en moyenne une fois aux 5 ans.
2. Estimez un intervalle de confiance à 95 % pour cette valeur.
3. Comparez cette estimation au quantile empirique.

In [5]:
data = CSV.read("tasmax.csv", DataFrame)
filter!(row -> year(row.Date) > 1941, data)
last(data, 5)

Row,Longitude,Latitude,Name,ID,Date,tasmax
,Float64,Float64,String,Int64,Date,Float64?
1,-73.74,45.47,MONTREAL INTL A,7025251,2024-12-27,-6.3
2,-73.74,45.47,MONTREAL INTL A,7025251,2024-12-28,-2.5
3,-73.74,45.47,MONTREAL INTL A,7025251,2024-12-29,7.8
4,-73.74,45.47,MONTREAL INTL A,7025251,2024-12-30,11.2
5,-73.74,45.47,MONTREAL INTL A,7025251,2024-12-31,6.0


# Exercice 4 - Pertes journalières de l'action S&P 500

Le S&P 500 est un indice boursier basé sur 500 grandes sociétés cotées sur les bourses aux États-Unis.

Le fichier SPX.csv contient les valeurs du titre à la fermeture, à l'ouverture, ainsi que le maximum et le minimum de la valeur dans la journée.

À partir de ces données, 

1. Calculez les pertes journalières, soit la valeur à l'ouverture - la valeur à la fermeture.
2. Estimez la perte journalière qui a une probabilité de 1/20 d'être excédée.
3. Estimez un intervalle de confiance à 95 % pour cette valeur.

In [6]:
data = CSV.read("SPX.csv", DataFrame)

data.Date = Date.(data.Date, DateFormat("m/d/y"))

last(data, 5)

Row,Date,Close/Last,Open,High,Low
,Date,Float64,Float64,Float64,Float64
1,2015-02-20,2110.3,2097.65,2110.61,2085.44
2,2015-02-19,2097.45,2099.25,2102.13,2090.79
3,2015-02-18,2099.68,2099.16,2100.23,2092.15
4,2015-02-17,2100.34,2096.47,2101.3,2089.8
5,2015-02-16,2096.99,0.0,0.0,0.0


In [7]:
data.DailyLoss = data[:, :Open] - data[:,Symbol("Close/Last")]
filter!(row -> row.Open >0., data)

last(data, 5)

Row,Date,Close/Last,Open,High,Low,DailyLoss
,Date,Float64,Float64,Float64,Float64,Float64
1,2015-02-23,2109.66,2109.83,2110.05,2103.0,0.17
2,2015-02-20,2110.3,2097.65,2110.61,2085.44,-12.65
3,2015-02-19,2097.45,2099.25,2102.13,2090.79,1.8
4,2015-02-18,2099.68,2099.16,2100.23,2092.15,-0.52
5,2015-02-17,2100.34,2096.47,2101.3,2089.8,-3.87
